In [1]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random
import time
import os
import math
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
#plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np
from nltk.translate.bleu_score import SmoothingFunction, sentence_bleu

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
def __save_model(model_name, model, root):
    if not os.path.isdir(root):
        os.mkdir(root)
    p = os.path.join(root, '{}-params.pkl'.format(model_name))
    torch.save(model.state_dict(), p)
    return p

def save_model(models, root='./model'):
    p = {}
    for k, m in models.items():
        p[k] = __save_model(k, m, root)
    return p

def __load_model(model_name, model, root):
    p = os.path.join(root, '{}-params.pkl'.format(model_name))
    if not os.path.isfile(p):
        msg = "No model parameters file for {}!".format(model_name)
        return print(msg)
        raise AttributeError(msg)
    paras = torch.load(p)
    model.load_state_dict(paras)

def load_model(models, root='./model'):
    for k, m in models.items():
        __load_model(k, m, root)
        
def save_model_by_score(models, loss, root):
    p = os.path.join(root, 'state.pkl')
    state = None
    
    if np.isnan(loss):
        raise AttributeError("Loss become {}".format(loss))
        return
    
    if os.path.isfile(p):
         state = torch.load(p)
            
    if state is not None and state['loss'] < loss:
        return;
    
    save_model(models, root)
    state = {'loss' : loss}
    torch.save(state, p)

In [4]:
class CharDict:
    def __init__(self):
        self.word2index = {}
        self.index2word = {}
        self.n_words = 0
        
        for i in range(26):
            self.addWord(chr(ord('a') + i))
        
        tokens = ["SOS", "EOS"]
        for t in tokens:
            self.addWord(t)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.index2word[self.n_words] = word
            self.n_words += 1

    def longtensorFromString(self, s):
        s = ["SOS"] + list(s) + ["EOS"]
        return torch.LongTensor([self.word2index[ch] for ch in s])
    
    def stringFromLongtensor(self, l, show_token=False, check_end=True):
        s = ""
        for i in l:
            ch = self.index2word[i.item()]
            if len(ch) > 1:
                if show_token:
                    __ch = "<{}>".format(ch)
                else:
                    __ch = ""
            else:
                __ch = ch
            s += __ch
            if check_end and ch == "EOS":
                break
        return s

class wordsDataset(Dataset):
    def __init__(self, train=True):
        if train:
            f = './train.txt'
        else:
            f = './test.txt'
        self.datas = np.loadtxt(f, dtype=np.str)
        
        if train:
            self.datas = self.datas.reshape(-1)
        else:
            '''
            sp -> p
            sp -> pg
            sp -> tp
            sp -> tp
            p  -> tp
            sp -> pg
            p  -> sp
            pg -> sp
            pg -> p
            pg -> tp
            '''
            self.targets = np.array([
                [0, 3],
                [0, 2],
                [0, 1],
                [0, 1],
                [3, 1],
                [0, 2],
                [3, 0],
                [2, 0],
                [2, 3],
                [2, 1],
            ])
        
        #self.tenses = ['sp', 'tp', 'pg', 'p']
        self.tenses = [
            'simple-present', 
            'third-person', 
            'present-progressive', 
            'simple-past'
        ]
        self.chardict = CharDict()
        
        self.train = train
    
    def __len__(self):
        return len(self.datas)
    
    def __getitem__(self, index):
        if self.train:
            c = index % len(self.tenses)
            return self.chardict.longtensorFromString(self.datas[index]), c
        else:
            i = self.chardict.longtensorFromString(self.datas[index, 0])
            ci = self.targets[index, 0]
            o = self.chardict.longtensorFromString(self.datas[index, 1])
            co = self.targets[index, 1]
            
            return i, ci, o, co

In [5]:
#Encoder
class EncoderRNN(nn.Module):
    def __init__(
        self, word_size, hidden_size, latent_size, 
        num_condition, condition_size
    ):
        super(EncoderRNN, self).__init__()
        self.word_size = word_size
        self.hidden_size = hidden_size
        self.condition_size = condition_size
        self.latent_size = latent_size

        self.condition_embedding = nn.Embedding(num_condition, condition_size)
        self.word_embedding = nn.Embedding(word_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.mean = nn.Linear(hidden_size, latent_size)
        self.logvar = nn.Linear(hidden_size, latent_size)

    def forward(self, inputs, init_hidden, input_condition):
        c = self.condition(input_condition)
        
        # get (1,1,hidden_size)
        hidden = torch.cat((init_hidden, c), dim=2)
        
        # get (seq, 1, hidden_size)
        x = self.word_embedding(inputs).view(-1, 1, self.hidden_size)
        
        # get (seq, 1, hidden_size), (1, 1, hidden_size)
        outputs, hidden = self.gru(x, hidden)
        
        # get (1, 1, hidden_size)
        m = self.mean(hidden)
        logvar = self.logvar(hidden)
        
        z = self.sample_z() * torch.exp(logvar/2) + m
        
        #self.m = m
        #self.logvar = logvar
        
        return z, m, logvar

    def initHidden(self):
        return torch.zeros(
            1, 1, self.hidden_size - self.condition_size, 
            device=device
        )
    
    def condition(self, c):
        c = torch.LongTensor([c]).to(device)
        return self.condition_embedding(c).view(1,1,-1)
    
    def sample_z(self):
        return torch.normal(
            torch.FloatTensor([0]*self.latent_size), 
            torch.FloatTensor([1]*self.latent_size)
        ).to(device)

# TODO Teacher forcing

In [57]:
#Decoder
class DecoderRNN(nn.Module):
    def __init__(
        self, word_size, hidden_size, latent_size, condition_size
    ):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.word_size = word_size

        self.latent_to_hidden = nn.Linear(
            latent_size+condition_size, hidden_size
        )
        self.word_embedding = nn.Embedding(word_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, word_size)
        
    def forward(self, inputs, z, c, teacher=False, hidden=None):
        # get (1,1,latent_size + condition_size)
        latent = torch.cat((z, c), dim=2)
        
        # get (1,1,hidden_size)
        if hidden is None:
            hidden = self.latent_to_hidden(latent)
            #print("get hidden from latent")
        
        # get (seq, 1, hidden_size)
        x = self.word_embedding(inputs).view(-1, 1, self.hidden_size)
        
        input_length = x.size(0)
        
        # get (seq, 1, hidden_size), (1, 1, hidden_size)
        if teacher:
            outputs = []
            for i in range(input_length-1):
                output, hidden = self.gru(x[i:i+1], hidden)
                hidden = x[i+1:i+2]
                outputs.append(output)
            
            outputs = torch.cat(outputs, dim=0)
        else:
            # Omit EOS token
            x = x[:-1]
            outputs, hidden = self.gru(x, hidden)
            
        # get (seq, word_size)
        outputs = self.out(outputs).view(-1, self.word_size)
        
        return outputs, hidden

In [29]:
# config

train_dataset = wordsDataset()
test_dataset = wordsDataset(False)

word_size = train_dataset.chardict.n_words
num_condition = len(train_dataset.tenses)
hidden_size = 256
latent_size = 32
condition_size = 8

teacher_forcing_ratio = 0.5
empty_input_ratio = 0.1
KLD_weight = 0.0
LR = 0.05

In [58]:
encoder = EncoderRNN(
    word_size, hidden_size, latent_size, num_condition, condition_size
).to(device)
decoder = DecoderRNN(
    word_size, hidden_size, latent_size, condition_size
).to(device)
encoder, decoder

(EncoderRNN(
   (condition_embedding): Embedding(4, 8)
   (word_embedding): Embedding(28, 256)
   (gru): GRU(256, 256)
   (mean): Linear(in_features=256, out_features=32, bias=True)
   (logvar): Linear(in_features=256, out_features=32, bias=True)
 ), DecoderRNN(
   (latent_to_hidden): Linear(in_features=40, out_features=256, bias=True)
   (word_embedding): Embedding(28, 256)
   (gru): GRU(256, 256)
   (out): Linear(in_features=256, out_features=28, bias=True)
 ))

In [59]:
load_model(
    {'encoder':encoder, 'decoder':decoder}, 
    os.path.join('.', 'temp')
)

# TODO KL weights

loss = cross_entropy + (kl w)*KL($q(Z|X, c;\theta') || p(Z|c)$)

In [10]:
def KLD_weight_annealing(*args):
    epoch, batch = args
    slope = 0.001
    #slope = 0.1
    scope = (1.0 / slope)*2
    
    w = (epoch % scope) * slope
    
    if w > 1.0:
        w = 1.0
    
    return w

In [11]:
def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '{:4d}m {:2d}s'.format(int(m), int(s))


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

def KL_loss(m, logvar):
    return torch.sum(0.5 * (-logvar + (m**2) + torch.exp(logvar) - 1))

def trainEpochs(
    name, encoder, decoder, epoch_size, learning_rate=1e-2,
    show_size=1000, KLD_weight=0.0, teacher_forcing_ratio = 1.0
):
    start = time.time()
    plot_losses = []
    show_loss_total = 0
    plot_loss_total = 0
    plot_kl_loss_total = 0
    char_accuracy_total = 0
    char_accuracy_len = 0

    encoder.train()
    decoder.train()
    
    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    
    criterion = nn.CrossEntropyLoss(reduction='sum')

    for epoch in range(epoch_size):
        # get data from trian dataset
        for idx in range(len(train_dataset)):   
        #for idx in range(1):
            data = train_dataset[idx]
            inputs, c = data
            
            encoder_optimizer.zero_grad()
            decoder_optimizer.zero_grad()
            
            # input no sos and eos
            z, m, logvar = encoder(inputs[1:-1].to(device), encoder.initHidden(), c)
            
            # decide teacher forcing
            use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False
            
            # input has sos
            outputs, _ = decoder(inputs.to(device), z, encoder.condition(c), use_teacher_forcing)
            
            # target no sos
            loss = criterion(outputs, inputs[1:].to(device))
            kld_loss = KL_loss(m, logvar)
            #loss = criterion(outputs, inputs[:-1].to(device))
            if callable(KLD_weight):
                kld_w = KLD_weight(epoch, idx)
            else:
                kld_w = KLD_weight
                
            #print('crossentropy : {} , kld : {}'.format(loss.item(), kld_loss.item()))
                
            (loss + (kld_w * kld_loss)).backward()
            
            encoder_optimizer.step()
            decoder_optimizer.step()
            
            show_loss_total += loss.item() + ( kld_w*kld_loss.item() )
            plot_loss_total += loss.item()
            plot_kl_loss_total += kld_loss.item()
            
            # show output by string
            # outputs_onehot = torch.max(outputs, 1)[1]
            outputs_onehot = torch.max(torch.softmax(outputs, dim=1), 1)[1]
            inputs_str = train_dataset.chardict.stringFromLongtensor(inputs, show_token=True)
            outputs_str = train_dataset.chardict.stringFromLongtensor(outputs_onehot, show_token=True)
            #print(inputs_str,':',outputs_str)
            
            char_accuracy_total += (outputs_onehot[:-1] == inputs[1:-1].to(device)).sum().item()
            char_accuracy_len += len(inputs[1:-1])
        
        if (epoch + 1)%show_size == 0:
            show_loss_total /= show_size
            print("{} ({} {}%) {:.4f}".format(
                timeSince(start, (epoch+1) / epoch_size),
                epoch+1, (epoch+1)*100/epoch_size, show_loss_total
            ))
            print('accuracy : {}'.format(char_accuracy_total/char_accuracy_len))
            show_loss_total = 0
            
        plot_losses.append((plot_loss_total, plot_kl_loss_total))
        
        if callable(KLD_weight):
            kld_w = KLD_weight(epoch, idx)
        else:
            kld_w = KLD_weight
        
        #print(epoch, kld_w)
        
        save_model_by_score(
            {'encoder':encoder, 'decoder':decoder}, 
            plot_loss_total + kld_w * plot_kl_loss_total, 
            os.path.join('.', name)
        )
        
        plot_loss_total = 0
        plot_kl_loss_total = 0
        char_accuracy_total = 0
        char_accuracy_len = 0
        
    return plot_losses

In [ ]:
%%time
trainEpochs('result', encoder, decoder, 500, show_size=10, KLD_weight=KLD_weight_annealing, teacher_forcing_ratio=0.5)

In [60]:
save_model({'encoder':encoder, 'decoder':decoder}, 
    os.path.join('.', 'best'))

{'encoder': './best/encoder-params.pkl',
 'decoder': './best/decoder-params.pkl'}

In [12]:
#compute BLEU-4 score
def compute_bleu(output, reference):
    cc = SmoothingFunction()
    return sentence_bleu(
        [reference], output,
        weights=(0.25, 0.25, 0.25, 0.25),smoothing_function=cc.method1
    )

In [13]:
def evaluation(encoder, decoder, dataset):
    encoder.eval()
    decoder.eval()
    
    char_accuracy_total = 0
    char_accuracy_len = 0
    
    blue_score = []
    
    for idx in range(len(dataset)):
        data = dataset[idx]
        if dataset.train:
            inputs, input_condition = data
            targets = inputs
            target_condition = input_condition
        else:
            inputs, input_condition, targets, target_condition = data
            
        # input no sos and eos
        z, _, _ = encoder(inputs[1:-1].to(device), encoder.initHidden(), input_condition)
            
        # input has sos
        outputs, _ = decoder(targets.to(device), z, encoder.condition(target_condition), False)
            
        # show output by string
        outputs_onehot = torch.max(torch.softmax(outputs, dim=1), 1)[1]
        targets_str = train_dataset.chardict.stringFromLongtensor(targets, check_end=True)
        outputs_str = train_dataset.chardict.stringFromLongtensor(outputs_onehot, check_end=True)
        
        #print(targets_str,':',outputs_str)
            
        char_accuracy_total += (outputs_onehot[:-1] == targets[1:-1].to(device)).sum().item()
        char_accuracy_len += len(targets[1:-1])
        
        blue_score.append( compute_bleu(outputs_str, targets_str) )
    
    print('Accuracy per char : {}'.format(char_accuracy_total / char_accuracy_len))
    print('BLEU-4 score : {}'.format(sum(blue_score) / len(blue_score)))
    
    return blue_score

In [14]:
all_score = evaluation(encoder, decoder, train_dataset)

Accuracy per char : 0.8205898268398268
BLEU-4 score : 0.5458485626650489


In [63]:
def generate_word(encoder, decoder, z, condition, maxlen=20):
    encoder.eval()
    decoder.eval()
    z = z.view(1,1,-1)
    sos_token = train_dataset.chardict.word2index['SOS']
    eos_token = train_dataset.chardict.word2index['EOS']
    inputs = torch.LongTensor([sos_token, eos_token])
    outputs = []
    i = 0
    hidden = None
    
    while True:
        # get (1, word_size)
        output, hidden = decoder(
            inputs.to(device), 
            z.to(device), 
            encoder.condition(condition),
            False,
            hidden
        )
        output_onehot = torch.max(torch.softmax(output, dim=1), 1)[1]
        if output_onehot.item() == eos_token:
            break
        
        outputs.append(output_onehot.item())
        i += 1
        if maxlen <= i:
            break
        
        inputs = torch.LongTensor([outputs[-1], eos_token])
        
    return torch.LongTensor(outputs)

In [72]:
noise = encoder.sample_z()

In [71]:
print(noise)
for i in range(len(train_dataset.tenses)):
    outputs = generate_word(encoder, decoder, noise, i)
    output_str = train_dataset.chardict.stringFromLongtensor(outputs)
    print('{:20s} : {}'.format(train_dataset.tenses[i],output_str))

tensor([-1.3527,  0.1130,  0.5260, -0.5226,  0.2138, -1.8048, -0.4402,  0.9119,
         0.5118, -1.7090, -0.7055,  0.5366,  0.5434,  0.9837, -0.0902, -1.7488,
        -0.6951, -0.2378, -1.1977, -0.2207,  0.7286, -1.0850, -0.5777,  0.3871,
        -0.8316,  1.4539, -0.3098, -1.2312,  0.2934,  1.2168, -0.9046,  0.0234],
       device='cuda:0')
simple-present       : blooze
third-person         : blooks
present-progressive  : blooming
simple-past          : bloomed


In [74]:
print(noise)
for i in range(len(train_dataset.tenses)):
    outputs = generate_word(encoder, decoder, noise, i)
    output_str = train_dataset.chardict.stringFromLongtensor(outputs)
    print('{:20s} : {}'.format(train_dataset.tenses[i],output_str))

tensor([-0.2851,  1.2386,  1.0263,  1.1914, -1.4874, -1.2694,  0.6261, -0.4220,
        -0.6354,  0.1487, -2.7937, -1.4543,  0.1237,  0.7397, -1.0753,  0.2940,
         0.0773, -0.9082,  1.0397, -0.0071,  0.0673, -0.0568, -0.3548,  0.2373,
        -0.2261, -0.8434,  0.6118,  0.2808,  1.0806, -0.1785,  0.0830, -0.6146],
       device='cuda:0')
simple-present       : throw
third-person         : throws
present-progressive  : throwing
simple-past          : throwed
